In [11]:
import tensorflow as tf
import os
import numpy as np
import ujson as json


from func import cudnn_gru, native_gru, dot_attention, summ, ptr_net
from prepro import word_tokenize, convert_idx
from inference import InfModel, Inference

In [12]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Must be consistant with training
char_limit = 16
hidden = 75
char_dim = 8
char_hidden = 100
use_cudnn = False

# File path
target_dir = "data"
save_dir = "log/model"
word_emb_file = os.path.join(target_dir, "word_emb.json")
char_emb_file = os.path.join(target_dir, "char_emb.json")
word2idx_file = os.path.join(target_dir, "word2idx.json")
char2idx_file = os.path.join(target_dir, "char2idx.json")

infer = Inference()

In [6]:
context = "sen. charles e. schumer called on federal safety officials yesterday to reopen " \
          "their investigation into the fatal crash of a passenger jet in belle_harbor , " \
          "queens , because equipment failure , not pilot error , might have been the cause "
ques1 = "Where is queens locations in?"
ans1 = infer.response(context, ques1)
print("Answer 1: {}".format(ans1))

context = "... michael crawford , right , who is ailing , will not return to his award-winning " \
          "role of count fosco in the andrew lloyd webber musical adaptation of the wilkie_collins " \
          "classic , '' the woman in white , '' in london as scheduled on may 2 ."
ques2 = "Where is the birth place of wilkie_collins?"
ans2 = infer.response(context, ques2)
print("Answer 2: {}".format(ans2))

Answer 1: belle_harbor
Answer 2: london


In [59]:
ques2 = "Where is the birth place of wilkie_collins?"
ans2 = infer.response(context, ques2)
print("Answer 2: {}".format(ans2))

Answer 2: london


In [7]:
file_path = 'origin_data/train.txt'

In [18]:
with open(file_path, 'r') as f:
    for line in f.readlines():
        line = line[:-10].split('\t', 2)[-1]
        e1, line = line.split('\t', 1)
        relaiton, line = line.split('\t', 1)
        e2, line = line.split('\t', 1)
        print((e1, relaiton, e2))
        print(line)
        
        break

('queens', 'belle_harbor', '/location/location/contains')
sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor , queens , because equipment failure , not pilot error , might have been the cause . 


In [19]:
import pandas as pd

In [22]:
df = pd.read_csv(file_path, sep='\t', header=None, names=['e1_encoding', 'e2_encoding', 'e1', 'e2', 'relation', 'content'])

C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df.relation = df.relation.fillna('none')
relation_series = df.relation.value_counts()
selected_relation_series = relation_series[relation_series.values > 1000]
relation_list = selected_relation_series.index.values.tolist()

In [49]:
relation_list

['none',
 '/location/location/contains',
 '/people/person/nationality',
 '/location/country/capital',
 '/people/person/place_lived',
 '/location/neighborhood/neighborhood_of',
 '/location/country/administrative_divisions',
 '/location/administrative_division/country',
 '/business/person/company',
 '/people/person/place_of_birth',
 '/people/deceased_person/place_of_death',
 '/business/company/founders']

In [ ]:
# 最好是输出对应位置的概率


In [50]:
df.loc[df['relation'].isin(relation_list)]

,e1_encoding,e2_encoding,e1,e2,relation,content
0,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,sen. charles e. schumer called on federal safe...
1,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,"but instead there was a funeral , at st. franc..."
2,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,"rosemary antonelle , the daughter of teresa l...."
3,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,one was for st. francis de sales roman catholi...
4,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,"the firefighter , whom a fire department offic..."
5,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,in st. francis de sales roman catholic church ...
6,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,"on nov. 12 , while walking his dog near his ho..."
7,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,"colm j. neilson , of belle_harbor , queens , s..."
8,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,she is a daughter of marion i. rabbin and dr. ...
9,m.0ccvx,m.05gf08,queens,belle_harbor,/location/location/contains,he is a son of vera and william lichtenberg of...
